### Copyright 2024 Lehi Gracia

### Markets

In [69]:

from yfinance_utils import list_utils, file_utils, constants
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pprint import pprint

symbols = list_utils.get_markets()
names = list_utils.markets
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="MARKETS", title_x=0.5)

gains = {}
for symbol in symbols:
    df1 = pd.read_csv(f'../../{constants.DATA_FOLDER}/{symbol}')
    df1 = df1[-250:]
    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_60 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round((float(temp_1 / temp_60)-1)*100,2)
fig.show()

print("CHANGE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:4}:{gains[names[symbol]]:8}: {names[symbol]}')



CHANGE IN THE LAST 30 DAYS:
SPY :    0.63: SNP 500
QQQ :    4.23: NASDAQ 100
DIA :   -2.43: DOW JONES


### Sectors

In [65]:

symbols = list_utils.get_sectors()
names = list_utils.sectors
 
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="SECTORS", title_x=0.5)

gains = {}
for symbol in symbols:
    df1 = pd.read_csv(f'../../{constants.DATA_FOLDER}/{symbol}')
    df1 = df1[-250:]
    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_30 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round((float(temp_1 / temp_60)-1)*100,2)
fig.show()

print("CHANGE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:6}:{gains[names[symbol]]:8}: {names[symbol]}')


CHANGE IN THE LAST 30 DAYS:
XLF   :  -88.93: FINANCIALS
XLE   :  -80.05: ENERGY
XLV   :   -68.3: HEALTH CARE
XLU   :   -82.6: UTILITIES
XLK   :   -45.4: TECHNOLOGY
XLC   :   -77.5: COMMUNICATION SERVICES
XLY   :  -48.46: CONSUMER DISCRETIONARY
XLRE  :  -90.81: REAL ESTATE
XLI   :  -69.81: INDUSTRIALS
XLB   :  -80.94: MATERIALS
XLP   :  -82.32: CONSUMER STAPLES


### Ratings from at least 50 analysts, higher than 85% UP

In [62]:

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_colwidth', None)


client = MongoClient('mongodb://localhost:27017/')
db = client['bigdata']
collection = db.market

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')


res = collection.find(
                 {'DATE':today, 'TOTAL':{"$gt":40}, 'script':'daily_analysts_ratings'}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
print(data)


          DATE  TICK  UP  DOWN  HOLD    AVG  TOTAL
0   2025-01-07  AMZN  63     1     3  94.03     67
1   2025-01-07   WMT  38     1     3  90.48     42
2   2025-01-07  DDOG  40     0     6  86.96     46
3   2025-01-07  MSFT  52     0     5  91.23     57
4   2025-01-07  META  58     2     8  85.29     68
5   2025-01-07  NVDA  59     0     4  93.65     63
6   2025-01-07  AVGO  38     0     5  88.37     43
7   2025-01-07  AMZN  63     1     3  94.03     67
8   2025-01-07   WMT  38     1     3  90.48     42
9   2025-01-07  DDOG  40     0     6  86.96     46
10  2025-01-07  MSFT  52     0     5  91.23     57
11  2025-01-07  META  58     2     8  85.29     68
12  2025-01-07  NVDA  59     0     4  93.65     63
13  2025-01-07  AVGO  38     0     5  88.37     43


### 52 week lows and highs

In [63]:

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_gap_up_close_high']}}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
print(data)

          DATE  TICK     HIGH    CLOSE     OPEN  CLOSE -1
0   2025-01-07  MPWR   644.81   643.07   633.10    632.37
1   2025-01-07   ADI   220.27   219.79   217.47    215.33
2   2025-01-07   CNC    63.18    62.88    62.77     61.76
3   2025-01-07   WMT    91.94    91.70    91.06     90.83
4   2025-01-07  HUBB   441.07   440.55   437.77    433.76
5   2025-01-07    BK    78.49    78.13    78.08     77.78
6   2025-01-07   WFC    73.12    72.97    72.03     71.93
7   2025-01-07    IT   493.61   493.33   492.80    487.11
8   2025-01-07   BBY    85.57    85.18    84.64     84.06
9   2025-01-07   DGX   154.00   153.28   153.17    152.46
10  2025-01-07  META   632.10   631.49   619.32    611.83
11  2025-01-07   HPE    23.03    23.05    22.45     22.22
12  2025-01-07  DELL   125.77   125.58   122.16    121.81
13  2025-01-07  PYPL    89.97    89.55    88.30     88.02
14  2025-01-07   PEG    86.55    86.33    85.59     85.56
15  2025-01-07  ASML   778.39   774.99   762.07    751.06
16  2025-01-07

### Price drops and jumps

In [64]:

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_jump_price_down','daily_jump_price_up']}}, 
                 {'_id':0, 'script':0, 'timestamp':0, 'VOLUME':0  }
                 )

data = pd.DataFrame(res).round(2)
print(data)


         DATE  TICK   CLOSE  % PRICE JUMP  % VOLUME -1  % VOLUME -2
0  2025-01-07  QBTS   10.44         13.85       -41.96        43.43
1  2025-01-07   SMR   23.59          6.02       -36.14        45.23
2  2025-01-07  IONQ   51.28          6.17       -13.48       -17.48
3  2025-01-07    MU  103.40          8.19        -5.25       145.38
4  2025-01-07   NNE   32.33         15.30       -52.42        95.43
5  2025-01-07  INCY   73.04          5.17       -45.89        25.22
6  2025-01-07  OKLO   30.00          9.25       -50.45        12.47
7  2025-01-07  LULU  404.42          6.03       -29.79        67.53
8  2025-01-07  AXON  570.72         -5.57       -65.78       271.49
9  2025-01-07  SERV   19.49         -6.84       -25.82        36.30
